#Modelo de Machine Learning de Ponta a Ponta

Depois que apresentamos as soluções para melhorar o desempenho da empresa e recuperar uma parte do faturamento, estamos prontos para criar um Modelo de Machine Learning capaz de prever a demanda para os próximos dias, gerando valor para os parceiros e levando informações preciosas para que eles possam se preparar para o dia de trabalho.

In [ ]:
#import das bibliotecas
import pandas as pd
import numpy as np

## Análise de Estrutura

## Remoção de Outliers

In [ ]:
#Leitura do csv
df_orders = pd.read_csv("orders.csv")

In [ ]:
#Corte de outliers
df_orders = df_orders[(df_orders['order_amount'] >= 15) &
          (df_orders['order_amount'] <= 200)]

In [ ]:
#info
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333629 entries, 0 to 368998
Data columns (total 29 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   order_id                           333629 non-null  int64  
 1   store_id                           333629 non-null  int64  
 2   channel_id                         333629 non-null  int64  
 3   payment_order_id                   333629 non-null  int64  
 4   delivery_order_id                  333629 non-null  int64  
 5   order_status                       333629 non-null  object 
 6   order_amount                       333629 non-null  float64
 7   order_delivery_fee                 333629 non-null  float64
 8   order_delivery_cost                327275 non-null  float64
 9   order_created_hour                 333629 non-null  int64  
 10  order_created_minute               333629 non-null  int64  
 11  order_created_day                  333629 no

## Novas Features

In [ ]:
#converter order_moment_created para data
df_orders['order_moment_created'] = pd.to_datetime(df_orders['order_moment_created'])

<ipython-input-6-5a5a7be8dc62>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_orders['order_moment_created'] = pd.to_datetime(df_orders['order_moment_created'])


In [ ]:
#nova coluna com o dia da semana
df_orders['day_of_week'] = df_orders['order_moment_created'].dt.day_of_week

In [ ]:
df_orders['order_moment_created'].dt.day_name()

,order_moment_created
0,Friday
1,Friday
2,Friday
3,Friday
4,Friday
...,...
368994,Friday
368995,Friday
368996,Friday
368997,Friday


In [ ]:
#Análise da quantidade de horas que temos pedidos por dia
df_orders['order_created_hour'].value_counts().reset_index().sort_values('order_created_hour')

,order_created_hour,count
6,0,25126
12,1,4002
13,2,1003
15,3,713
18,4,267
17,5,417
19,6,260
21,7,214
23,8,124
22,9,126


In [ ]:
#criando função por faixa de horário
def faixa_horario(hora):
    if hora >= 0 and hora <= 5:
      return 'madrugada'
    elif hora >= 6 and hora <= 10:
      return 'manha'
    elif hora >= 11 and hora <= 14:
      return 'almoco'
    elif hora >= 15 and hora <= 18:
      return 'tarde'
    else:
      return 'noite'

faixa_horario(13)

'almoco'

In [ ]:
#aplicando função e criando uma nova coluna com o nome de faixa de horário
df_orders['faixa_horario'] = df_orders['order_created_hour'].apply(faixa_horario)

In [ ]:
#distribuição da faixa de horário
df_orders['faixa_horario'].value_counts()

,count
faixa_horario,
noite,144269
tarde,119598
almoco,37294
madrugada,31528
manha,940


## DataPrep

In [ ]:
#Criando um novo dataset de orders_treatment
df_orders_treatment = df_orders.copy()

In [ ]:
#info
df_orders_treatment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333629 entries, 0 to 368998
Data columns (total 31 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   order_id                           333629 non-null  int64         
 1   store_id                           333629 non-null  int64         
 2   channel_id                         333629 non-null  int64         
 3   payment_order_id                   333629 non-null  int64         
 4   delivery_order_id                  333629 non-null  int64         
 5   order_status                       333629 non-null  object        
 6   order_amount                       333629 non-null  float64       
 7   order_delivery_fee                 333629 non-null  float64       
 8   order_delivery_cost                327275 non-null  float64       
 9   order_created_hour                 333629 non-null  int64         
 10  order_created_minute     

### Data Cleaning

#### Exclusão de colunas

In [ ]:
#após realizar análise das variáveis excluir variável order_moment_delivered
df_orders_treatment.drop('order_moment_delivered', axis=1, inplace=True)

In [ ]:
#definição das colunas que serão excluídas da nossa base
columns_delete = ['payment_order_id',
'delivery_order_id',
'order_status',
'order_created_minute',
'order_created_month',
'order_created_year',
'order_moment_created',
'order_moment_accepted',
'order_moment_ready',
'order_moment_collected',
'order_moment_in_expedition',
'order_moment_delivering',
'order_moment_finished',
'order_delivery_fee',
'order_delivery_cost',
'order_created_hour']

In [ ]:
#excluir colunas
df_orders_treatment.drop(columns_delete, axis=1, inplace=True)

#### Prenchendo valores nulos

In [ ]:
df_orders_treatment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333629 entries, 0 to 368998
Data columns (total 14 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   order_id                           333629 non-null  int64  
 1   store_id                           333629 non-null  int64  
 2   channel_id                         333629 non-null  int64  
 3   order_amount                       333629 non-null  float64
 4   order_created_day                  333629 non-null  int64  
 5   order_metric_collected_time        290223 non-null  float64
 6   order_metric_paused_time           272423 non-null  float64
 7   order_metric_production_time       312151 non-null  float64
 8   order_metric_walking_time          269553 non-null  float64
 9   order_metric_expediton_speed_time  304793 non-null  float64
 10  order_metric_transit_time          312372 non-null  float64
 11  order_metric_cycle_time            320707 no

In [ ]:
#utilização do fillna com a mediana para preencher os valores do order_metric_collected_time
df_orders_treatment['order_metric_collected_time'].fillna(df_orders_treatment['order_metric_collected_time'].median(), inplace=True)

<ipython-input-19-ad6bf6223430>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_orders_treatment['order_metric_collected_time'].fillna(df_orders_treatment['order_metric_collected_time'].median(), inplace=True)


### Criação do dataset final

In [ ]:
#Criação do dataset final de grupos
df_orders_treatment_group = df_orders_treatment.groupby(['store_id', 'channel_id', 'order_created_day', 'day_of_week', 'faixa_horario']) \
  .agg({'order_metric_collected_time':'median',
        'order_metric_paused_time':'median',
        'order_metric_production_time':'median',
        'order_metric_walking_time':'median',
        'order_metric_expediton_speed_time':'median',
        'order_metric_transit_time':'median',
        'order_metric_cycle_time':'median',
        'order_id':'count'}) \
  .reset_index() \
  .sort_values('order_id', ascending=False)

df_orders_treatment_group=df_orders_treatment_group.rename(columns = {'order_id':'demanda'})

df_orders_treatment_group.head()

,store_id,channel_id,order_created_day,day_of_week,faixa_horario,order_metric_collected_time,order_metric_paused_time,order_metric_production_time,order_metric_walking_time,order_metric_expediton_speed_time,order_metric_transit_time,order_metric_cycle_time,demanda
22931,676,5,26,4,noite,1.600,2.930,30.135,2.300,5.950,15.320,52.150,182
22766,676,5,12,4,noite,1.600,3.475,34.250,1.840,6.250,14.185,57.880,157
22707,676,5,7,6,tarde,2.500,2.330,30.035,2.880,6.375,13.450,49.785,154
22956,676,5,28,6,tarde,1.600,2.875,33.520,1.640,5.170,12.470,52.760,147
24297,707,5,26,4,noite,1.345,1.570,21.525,4.465,6.075,16.025,48.625,146


In [ ]:
#info
df_orders_treatment_group.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66624 entries, 22931 to 66623
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   store_id                           66624 non-null  int64  
 1   channel_id                         66624 non-null  int64  
 2   order_created_day                  66624 non-null  int64  
 3   day_of_week                        66624 non-null  int32  
 4   faixa_horario                      66624 non-null  object 
 5   order_metric_collected_time        66624 non-null  float64
 6   order_metric_paused_time           57376 non-null  float64
 7   order_metric_production_time       62378 non-null  float64
 8   order_metric_walking_time          57516 non-null  float64
 9   order_metric_expediton_speed_time  61213 non-null  float64
 10  order_metric_transit_time          62402 non-null  float64
 11  order_metric_cycle_time            63752 non-null  floa

### Criando um conjunto de testes

Antes de seguir adiante, vamos precisar criar um conjunto de teste, colocá-lo de lado e nunca checá-lo. <br/>

Quando estimamos o erro de generalização utilizando o conjunto de teste, sua estimativa será muito otimista e será lançado um sistema que não funcionará tão bem quanto o esperado.  <br/>

Isso é chamado de **data snooping bias.** <br/>

O Scikit-Learn fornece algumas funções para dividir conjuntos de dados em vários subconjuntos de diversas maneiras. A função mais simples é train_test_split.

In [ ]:
#import do train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
#Separa a base de treino e teste
train_set, test_set = train_test_split(df_orders_treatment_group, test_size=0.2, random_state=42)
df_orders_train = train_set.copy()

In [ ]:
#Separa a Label principal
df_orders_treatment_label = df_orders_train[['demanda']].copy()

In [ ]:
#Remove a Label da base
df_orders_treatment = df_orders_train.drop('demanda', axis=1)

In [ ]:
#import do numpy
import numpy as np

In [ ]:
#Seleciona as variáveis float64
df_orders_float64 = df_orders_treatment.select_dtypes(np.float64).copy()

In [ ]:
#Import do SimpleImputer
from sklearn.impute import SimpleImputer

In [ ]:
#Adicionando estratégia da mediana no imputer
imputer = SimpleImputer(strategy="median")

In [ ]:
#Treina o imputer
imputer.fit(df_orders_float64)

SimpleImputer(strategy='median')

In [ ]:
#Cria nova matriz preenchendo os valores nulos com o imputer
float_vars =  imputer.transform(df_orders_float64)

In [ ]:
#Cria novo dataset para apresentar os valores
df_float64 = pd.DataFrame(float_vars, columns=df_orders_float64.columns,
                          index=df_orders_float64.index)
df_float64.head()

,order_metric_collected_time,order_metric_paused_time,order_metric_production_time,order_metric_walking_time,order_metric_expediton_speed_time,order_metric_transit_time,order_metric_cycle_time
49037,1.47,2.150,20.47,2.450,5.05,15.870,36.45
23762,3.20,3.035,18.80,4.155,7.60,18.695,48.59
43636,5.02,53.200,5.03,6.630,59.83,58.370,123.23
6376,2.30,0.970,20.37,6.820,7.80,10.020,41.18
288,0.67,2.270,14.62,0.800,3.05,12.820,32.67


In [ ]:
#info do novo dataset
df_float64.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53299 entries, 49037 to 9950
Data columns (total 7 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   order_metric_collected_time        53299 non-null  float64
 1   order_metric_paused_time           53299 non-null  float64
 2   order_metric_production_time       53299 non-null  float64
 3   order_metric_walking_time          53299 non-null  float64
 4   order_metric_expediton_speed_time  53299 non-null  float64
 5   order_metric_transit_time          53299 non-null  float64
 6   order_metric_cycle_time            53299 non-null  float64
dtypes: float64(7)
memory usage: 3.3 MB


## Seleção Final

## Variáveis Categóricas

In [ ]:
#info
df_orders_treatment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53299 entries, 49037 to 9950
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   store_id                           53299 non-null  int64  
 1   channel_id                         53299 non-null  int64  
 2   order_created_day                  53299 non-null  int64  
 3   day_of_week                        53299 non-null  int32  
 4   faixa_horario                      53299 non-null  object 
 5   order_metric_collected_time        53299 non-null  float64
 6   order_metric_paused_time           45895 non-null  float64
 7   order_metric_production_time       49878 non-null  float64
 8   order_metric_walking_time          45998 non-null  float64
 9   order_metric_expediton_speed_time  48955 non-null  float64
 10  order_metric_transit_time          49934 non-null  float64
 11  order_metric_cycle_time            51003 non-null  float

In [ ]:
#seleção de day_of_week e faixa_horario
df_orders_treatment[['day_of_week', 'faixa_horario']].head()

,day_of_week,faixa_horario
49037,5,tarde
23762,6,noite
43636,2,tarde
6376,0,almoco
288,2,noite


In [ ]:
#value_counts faixa de horario
df_orders_treatment['faixa_horario'].value_counts()

,count
faixa_horario,
noite,18451
tarde,18100
almoco,9048
madrugada,7493
manha,207


In [ ]:
#novo dataset com a seleção das variaveis numéricas categóricas
df_orders_treatment_cat_num = df_orders_treatment[['store_id', 'channel_id', 'order_created_day']].copy()

In [ ]:
#novo dataset com a seleção das variáveis categóricas
df_orders_treatment_cat = df_orders_treatment[['faixa_horario']]

In [ ]:
#import do OrdinalEncoder e do OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

### Ordinal Encoder
A maioria dos algoritmos de Aprendizado de Máquina prefere trabalhar com números, então vamos converter as categorias de texto para números. Para tanto, podemos utilizar o método OrdinalEncoder(), que mapeia cada categoria para um número inteiro diferente.

In [ ]:
#tratamento com o OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
df_orders_treatment_cat_encoded = ordinal_encoder.fit_transform(df_orders_treatment_cat)
df_orders_treatment_cat_encoded[:10]

array([[4.],
       [3.],
       [4.],
       [0.],
       [3.],
       [4.],
       [4.],
       [0.],
       [4.],
       [4.]])

In [ ]:
#Verificação das categorias do OrdinalEncoder
ordinal_encoder.categories_

[array(['almoco', 'madrugada', 'manha', 'noite', 'tarde'], dtype=object)]

### OneHotEncoder
O OrdinalEncoder pega a quantidade de atributos e converte em números, porém transformando puramente em números ele cria uma diferença de valores entre os números. As categorias 0 e 1 transformadas tem uma distância semelhante, não podemos dizer o mesmo para as categorias 0 e 2, os algoritmos de ML enxergarão essa escala como uma diferença significa entre os dados. <br/>
A utilização do OneHotEncoder é melhor aproveitada para esses casos. Ela cria novos atributos de acordo com a quantidade de atributos com 0 e 1.


In [ ]:
#tratamento da categoria com o OneHotEncoder
cat_encoder = OneHotEncoder()
df_orders_treatment_cat_1hot = cat_encoder.fit_transform(df_orders_treatment_cat)
df_orders_treatment_cat_1hot

<53299x5 sparse matrix of type '<class 'numpy.float64'>'
	with 53299 stored elements in Compressed Sparse Row format>

In [ ]:
#Visuaulização das categorias em array
df_orders_treatment_cat_1hot.toarray()

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.]])

In [ ]:
#Visuaulização das categorias
cat_encoder.categories_

[array(['almoco', 'madrugada', 'manha', 'noite', 'tarde'], dtype=object)]

## Escalonando nossos dados

In [ ]:
#visualização do df_float64
df_float64.head()

,order_metric_collected_time,order_metric_paused_time,order_metric_production_time,order_metric_walking_time,order_metric_expediton_speed_time,order_metric_transit_time,order_metric_cycle_time
49037,1.47,2.150,20.47,2.450,5.05,15.870,36.45
23762,3.20,3.035,18.80,4.155,7.60,18.695,48.59
43636,5.02,53.200,5.03,6.630,59.83,58.370,123.23
6376,2.30,0.970,20.37,6.820,7.80,10.020,41.18
288,0.67,2.270,14.62,0.800,3.05,12.820,32.67


In [ ]:
#import do MinMaxScaler e do StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

### MinMaxScaler
O escalonamento min-max (muitas pessoas chamam de normalização) é bastante simples: os valores são deslocados e redimensionados para que acabem variando de 0 a 1. Ele subtrai o valor mínimo e divide pelo máximo menos o mínimo. O Scikit-Learn fornece um transformador chamado MinMaxScaler para isso. Ele possui um hiper parâmetro feature_range que permite alterar o intervalo se não quiser 0-1 por algum motivo.

In [ ]:
#Tratamento com MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df_float64)
df_float64_transform = scaler.transform(df_float64)

In [ ]:
#Visualiza transformação
df_float64_transform

array([[1.06106463e-03, 8.90955871e-03, 3.29045984e-04, ...,
        4.30953559e-04, 3.35704995e-04, 5.53693404e-04],
       [2.29303899e-03, 8.98446096e-03, 3.02201490e-04, ...,
        6.48563773e-04, 3.86501279e-04, 7.40100446e-04],
       [3.58910450e-03, 1.32301911e-02, 8.08549730e-05, ...,
        5.10573296e-03, 1.09989687e-03, 1.88618130e-03],
       ...,
       [1.95121951e-03, 8.75721516e-03, 2.73267304e-05, ...,
        4.56554761e-04, 3.12509524e-04, 3.26135549e-04],
       [8.79473028e-04, 9.21593852e-03, 2.45940574e-04, ...,
        7.10006656e-04, 3.82095937e-04, 6.52578194e-04],
       [4.27274346e-04, 9.07036579e-03, 1.90965622e-04, ...,
        6.74164974e-04, 3.68250579e-04, 7.09083789e-04]])

### StandardScaler
A padronização é bem diferente: em primeiro lugar ela subtrai o valor médio (assim os valores padronizados sempre têm média zero) e, em seguida, divide pela variância, de modo que a distribuição resultante tenha variância unitária.
Ao contrário do escalonamento min-max, a padronização não vincula valores a um intervalo específico, o que pode ser um problema para alguns algoritmos.
No entanto, a padronização é muito menos afetada por outliers.
O Scikit-Learn fornece um transformador para padronização chamado StandardScaler.

In [ ]:
#Tratamento com o SatandardScaler
standard = StandardScaler()
standard.fit(df_float64)
df_float64_stardard_transform = standard.transform(df_float64)

In [ ]:
#Visualiza transformação
df_float64_stardard_transform

array([[-0.09533207, -0.08201388, -0.05910494, ..., -0.12363719,
        -0.0393002 , -0.1022982 ],
       [ 0.06297516, -0.07017721, -0.06320582, ..., -0.09488854,
        -0.03469955, -0.08777666],
       [ 0.22951802,  0.60076801, -0.09701967, ...,  0.49395144,
         0.02991317,  0.00150571],
       ...,
       [ 0.01905177, -0.10608846, -0.10519688, ..., -0.120255  ,
        -0.04140103, -0.12002549],
       [-0.11866637, -0.03359722, -0.07180048, ..., -0.08677128,
        -0.03509854, -0.09459485],
       [-0.17677336, -0.05660182, -0.08019869, ..., -0.09150635,
        -0.03635253, -0.09019293]])

## Pipeline

In [ ]:
#import do Pipeline
from sklearn.pipeline import Pipeline

Existem muitas etapas de transformação de dados que precisam ser executadas na ordem correta. Felizmente, o Scikit-Learn fornece a classe Pipeline para ajudar tais sequências de transformações.

O construtor Pipeline se vale de uma lista de pares de nome/estimador que definem uma sequência de etapas. Todos, exceto o último estimador, devem ser transformadores (ou seja, eles devem ter um método fit_transform()).


### Pipeline Numérico

In [ ]:
#Criação do novo pipeline com o Imputer e StandardScaler
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

In [ ]:
#Criação de um novo dataset com o pipline numérico
orders_num_tr = num_pipeline.fit_transform(df_orders_float64)

In [ ]:
#Apresentação do Shape da transformação após o pipeline
orders_num_tr.shape

(53299, 7)

### Pipeline Categório + Full Pipeline

In [ ]:
#Import do column transform
from sklearn.compose import ColumnTransformer

In [ ]:
#Seleção das variáveis numéricas, categóricas numéricas e categóricas.
num_attr = list(df_orders_float64.columns)
cat_num_attr = list(df_orders_treatment_cat_num.columns)
cat_attr = list(df_orders_treatment_cat.columns)

In [ ]:
#Criação do Full Pipeline
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attr),
    ("cat_num", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_num_attr),
    ("cat", OneHotEncoder(handle_unknown='ignore'), cat_attr),
])

In [ ]:
#Transformação dos dados através do full pipeline
df_orders_prepared = full_pipeline.fit_transform(df_orders_treatment)

In [ ]:
#Apresentação do shspe dos dados transformados
df_orders_prepared.shape

(53299, 15)

## Trienando o modelo

### Regressão Linear
A Regressão Linear é uma técnica estatística usada para modelar a relação entre uma variável dependente (ou alvo) e uma ou mais variáveis independentes (ou preditoras). O objetivo é ajustar uma linha reta (ou plano, no caso de múltiplas variáveis) que melhor represente essa relação, de forma a prever os valores da variável dependente com base nos valores das variáveis independentes.

**Regressão Linear Simples:**<br/>
Utiliza uma única variável independente para prever o valor da variável dependente. A equação da reta é dada por:

$$
y = \beta_0 + \beta_1 x + \epsilon
$$

Onde:
- $( y  )$ é a variável dependente (o que você quer prever),
- \( $\beta_0 $ ) é o intercepto (onde a linha cruza o eixo y),
- \( $\beta_1 $) é o coeficiente angular (influência de \( x \) sobre \( y \)),
- \( x \) é a variável independente (o que você usa para prever),
- \( $\epsilon \$ ) é o erro ou termo de resíduo.

**Regressão Linear Múltipla:** <br/>
Utiliza duas ou mais variáveis independentes para prever o valor da variável dependente. A equação se expande para:

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_n x_n + \epsilon
$$



<img src="https://analisemacro.com.br/wp-content/uploads/2023/08/p3.png">

In [ ]:
#import e modelo de regressão linear
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(df_orders_prepared, df_orders_treatment_label)

LinearRegression()

In [ ]:
#Seleciona os dados e as labels e passa pelo full pipeline
dados = df_orders_treatment.iloc[:10]
labels = df_orders_treatment_label.iloc[:10]

dados_preparados = full_pipeline.transform(dados)

In [ ]:
#Verifica as predições
print("Predições:", lin_reg.predict(dados_preparados))

Predições: [[ 4.56701953]
 [ 7.44404133]
 [ 5.08456469]
 [ 5.14307922]
 [ 8.18394773]
 [ 5.39074209]
 [ 6.63937496]
 [-0.35757434]
 [ 5.44404623]
 [ 5.8924963 ]]


In [ ]:
#Verifica os valores reais
print("Reais:", labels.values)

Reais: [[7]
 [8]
 [1]
 [3]
 [9]
 [1]
 [3]
 [5]
 [3]
 [3]]


### RMSE (Root Mean Squared Error)

O **RMSE (Root Mean Squared Error)** ou **Erro Quadrático Médio da Raiz** é uma métrica usada para avaliar a qualidade de um modelo de regressão, especialmente em Machine Learning. Ele mede a diferença entre os valores preditos por um modelo e os valores reais observados. O RMSE dá uma indicação de quão bem o modelo está prevendo os dados.

**Como é calculado o RMSE?** <br/>

O RMSE é calculado em três etapas:

1. **Calcula-se o erro para cada ponto de dados**: A diferença entre o valor observado (real) e o valor previsto pelo modelo.
   
   $$ \text{Erro} = y_{\text{real}} - y_{\text{previsto}} $$

2. **Eleva-se o erro ao quadrado**: Isso é feito para garantir que os erros negativos não cancelem os positivos, e também para dar maior peso a erros maiores.
$$ \text{Erro Quadrático} = (y_{\text{real}} - y_{\text{previsto}})^2 $$

3. **Calcula-se a média dos erros quadráticos**: Soma-se todos os erros quadráticos e divide-se pelo número total de amostras.

   $$ \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_{\text{real}}^{(i)} - y_{\text{previsto}}^{(i)})^2 $$

4. **Tira-se a raiz quadrada** da média dos erros quadráticos:

   $$\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_{\text{real}}^{(i)} - y_{\text{previsto}}^{(i)})^2}$$

**Interpretação do RMSE:**
- **RMSE = 0**: Indica que o modelo fez previsões perfeitas, ou seja, não houve erro entre os valores previstos e os reais.
- **Valores maiores de RMSE**: Sugerem que o modelo tem maior erro de previsão. Quanto maior o RMSE, pior o modelo se ajusta aos dados.

**Vantagens do RMSE:**
- **Penaliza erros grandes**: Como o erro é elevado ao quadrado, os erros maiores têm um impacto mais significativo na métrica, o que é útil para identificar modelos que cometem grandes erros.

**Limitação:**
- **Sensível a outliers**: Como o erro é elevado ao quadrado, o RMSE é particularmente sensível a outliers (valores muito distantes dos demais), o que pode distorcer a avaliação do modelo.

**Exemplo:**
Se um modelo de regressão está prevendo o preço de casas, o RMSE pode ser usado para avaliar o quão próxima a previsão de preço está em relação ao preço real das casas. Se o RMSE for baixo, significa que as previsões estão próximas dos valores reais; se for alto, indica que as previsões estão dispersas dos valores reais.
<br/><br/>
**Comparação com outras métricas:**
- O RMSE pode ser comparado com o **MAE (Mean Absolute Error)**, que é outra métrica comum. A diferença principal é que o RMSE penaliza mais fortemente os erros grandes, enquanto o MAE trata todos os erros de forma linear.


In [ ]:
#Import da métrica erro quadrado médio
from sklearn.metrics import mean_squared_error

dados_predictions = lin_reg.predict(dados_preparados)
lin_mse = mean_squared_error(labels, dados_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

3.2208391514032813

### MAE (Mean Absolute Error)

O **MAE (Mean Absolute Error)** ou **Erro Médio Absoluto** é uma métrica de avaliação usada em modelos de regressão. Ele mede a magnitude média dos erros entre os valores previstos pelo modelo e os valores reais, sem considerar a direção do erro (se o valor previsto foi maior ou menor que o valor real).

## Fórmula do MAE:

A fórmula para calcular o MAE é:

$$
MAE = \frac{1}{n} \sum_{i=1}^{n} |y_{\text{real}}^{(i)} - y_{\text{previsto}}^{(i)}|
$$

**Como o MAE funciona:**
- O MAE simplesmente soma a magnitude de todos os erros (diferença entre previsto e real) e divide pelo número total de pontos de dados.
- Como utiliza o valor absoluto, o MAE não se preocupa se o modelo está subestimando ou superestimando as previsões, apenas com a magnitude média do erro.

**Interpretação do MAE:**
- **MAE = 0**: Significa que o modelo fez previsões perfeitas, ou seja, não há diferença entre os valores reais e os previstos.
- **Valores maiores de MAE**: Indicam que as previsões do modelo estão, em média, mais distantes dos valores reais.

**Vantagens do MAE:**
- **Fácil de interpretar**: O MAE representa o erro médio em unidades da variável prevista, facilitando a compreensão do quão distantes estão as previsões em relação aos valores reais.
- **Robustez**: O MAE é menos sensível a outliers (valores extremos) comparado ao RMSE, já que não eleva os erros ao quadrado.

**Limitações do MAE:**
- **Não penaliza grandes erros com intensidade**: Como não eleva os erros ao quadrado (diferente do RMSE), o MAE trata todos os erros de forma igual. Portanto, grandes erros não são penalizados de forma mais acentuada, o que pode ser uma desvantagem em alguns contextos.

**Exemplo de uso do MAE:**
Se você estiver prevendo o preço de casas, o MAE pode indicar o erro médio em dólares entre as previsões do modelo e os preços reais. Por exemplo, um **MAE de 5.000 dólares** significa que, em média, o modelo erra em 5.000 dólares ao prever o preço de uma casa.

**Comparação com RMSE:**
- **MAE** trata todos os erros de forma linear, sem dar mais peso aos erros maiores.
- **RMSE** penaliza mais fortemente os grandes erros porque eleva os erros ao quadrado antes de calcular a média.

Ambas as métricas são úteis, e a escolha entre uma e outra depende do tipo de análise que você deseja fazer. O **MAE** é mais simples e intuitivo, enquanto o **RMSE** é mais sensível a grandes erros.


In [ ]:
#Import da métrica MAE
from sklearn.metrics import mean_absolute_error

# Gera as previsões
dados_predictions = lin_reg.predict(dados_preparados)

# Calcula o MAE
lin_mae = mean_absolute_error(labels, dados_predictions)
lin_mae


2.8756869243662093

In [ ]:
#Describe dos Labels
labels['demanda'].describe()

,demanda
count,10.000000
mean,4.300000
std,2.830391
min,1.000000
25%,3.000000
50%,3.000000
75%,6.500000
max,9.000000


### R²



# Coeficiente de Determinação (R²)

O **R²** (Coeficiente de Determinação) é uma métrica estatística utilizada para avaliar a qualidade de ajuste de um modelo de regressão. Ele mede a proporção da variabilidade total da variável dependente que é explicada pelas variáveis independentes do modelo.

## Interpretação do R²:
- **R² = 1**: O modelo explica 100% da variabilidade dos dados, ou seja, ajuste perfeito.
- **R² = 0**: O modelo não explica nenhuma variabilidade dos dados. Nesse caso, a média dos dados seria um preditor tão bom quanto o modelo.
- **R² < 0**: O modelo pode ser pior do que simplesmente usar a média dos dados como predição.

## Fórmula do R²:

$$
R² = 1 - \frac{\text{SSR}}{\text{SST}}
$$

Onde:
- **SSR (Soma dos Quadrados dos Resíduos)**: Representa a soma das diferenças quadráticas entre os valores observados e os preditos pelo modelo.
- **SST (Soma Total dos Quadrados)**: Representa a soma das diferenças quadráticas entre os valores observados e a média dos valores observados.

Se a **SSR** for pequena, significa que o modelo faz boas previsões, e o valor de **R²** será próximo de 1.



In [ ]:
#import do R²
from sklearn.metrics import r2_score

In [ ]:
#Nova variáveil com o R²
r_squared = r2_score(labels, dados_predictions)

# Exibindo o resultado
print(f"R²: {r_squared}")

R²: -0.43880788338588217


### **Underfitting**
Acontece quando o modelo é muito simples e não consegue capturar a complexidade dos dados de treinamento. Como resultado, ele apresenta um desempenho ruim tanto nos dados de treinamento quanto nos dados de teste (ou novos dados), pois não consegue aprender adequadamente os padrões.

**Causas principais:**

- Modelo muito simples (ex.: uma linha reta para dados que requerem uma curva mais complexa).
- Poucos parâmetros ou features.
- Pouco tempo de treinamento.
- Alto erro nos dados de treinamento.
- Baixo desempenho em dados novos (de teste ou validação).

**Solução:**

- Aumentar a complexidade do modelo.
- Adicionar mais features relevantes.
- Reduzir a regularização, se estiver em uso.


## Árvore de Decisão (DecisionTree)

A Árvore de Decisão é um algoritmo de aprendizado supervisionado usado tanto para problemas de classificação quanto de regressão. Ela funciona como um conjunto de regras de decisão, onde os dados são divididos em subgrupos com base em características específicas. É chamada de "árvore" porque a estrutura se assemelha a um diagrama em forma de árvore, com um nó raiz no topo, nós internos que representam testes em atributos/características, e folhas que representam as decisões finais ou previsões.

Estrutura da Árvore de Decisão:
- **Nó raiz (Root Node):** É o nó inicial que contém o conjunto completo de dados e é o ponto de partida para a construção da árvore. Este nó representa a divisão do primeiro atributo.

- **Nós internos (Internal Nodes):** Cada nó interno representa um teste em um atributo. Aqui, os dados são divididos com base em uma condição de decisão (por exemplo, "Salário > 50K").

- **Galhos (Branches):** São os caminhos que conectam os nós. Cada ramo representa o resultado de um teste e direciona o fluxo de dados para os nós filhos.

- **Folhas (Leaf Nodes):** Os nós folha representam a classe ou valor final de previsão. No caso de classificação, cada folha terá uma classe; no caso de regressão, terá um valor numérico.

In [ ]:
#import da árvore de decisão
from sklearn.tree import DecisionTreeRegressor

In [ ]:
#Treinamento da Árvore de Decisão
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(df_orders_prepared, df_orders_treatment_label)

DecisionTreeRegressor(random_state=42)

In [ ]:
#Realiza as previsões e testa a métrica do RMSE
dados_predictions = tree_reg.predict(dados_preparados)
tree_mse = mean_squared_error(labels, dados_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

In [ ]:
#Dados Predidos
dados_predictions

array([7., 8., 1., 3., 9., 1., 3., 5., 3., 3.])

In [ ]:
#dados reais
labels.values

array([[7],
       [8],
       [1],
       [3],
       [9],
       [1],
       [3],
       [5],
       [3],
       [3]])

In [ ]:
#Nova variáveil com o R²
r_squared = r2_score(labels, dados_predictions)

# Exibindo o resultado
print(f"R²: {r_squared}")

R²: 1.0


### Overfitting
Acontece quando o modelo é muito complexo e se ajusta muito bem aos dados de treinamento, capturando até mesmo o ruído e as peculiaridades dos dados. Como resultado, o modelo tem um ótimo desempenho nos dados de treinamento, mas um desempenho ruim em dados novos, porque ele "aprendeu demais" sobre os dados de treinamento e não consegue generalizar.

**Causas principais:**

- Modelo muito complexo (ex.: muitas camadas em uma rede neural ou uma árvore de decisão muito profunda).
- Muito tempo de treinamento, levando o modelo a memorizar os dados de treinamento.
- Pequeno conjunto de dados de treinamento (poucos exemplos).

**Sintomas:**

- Baixo erro nos dados de treinamento.
- Alto erro nos dados de teste ou validação.

**Solução:**

- Obter mais dados para o treinamento.
- Reduzir a complexidade do modelo (ex.: reduzir o número de parâmetros).

## Árvores Aleatória (Randon Forest)

As Árvores Aleatórias (ou Random Forest) são uma extensão do algoritmo de Árvores de Decisão que combinam várias árvores de decisão para melhorar a precisão do modelo e reduzir o risco de overfitting (superajuste). Essa técnica é amplamente utilizada tanto para tarefas de classificação quanto de regressão e é conhecida por ser robusta, precisa e fácil de interpretar.

O que é uma Random Forest?
Uma Random Forest é composta por um conjunto de árvores de decisão independentes (daí o nome "floresta"), onde cada árvore é treinada em uma amostra diferente dos dados e com um subconjunto aleatório dos atributos. As previsões finais são feitas com base no consenso das árvores, o que reduz a variância e melhora a generalização do modelo.

- Para classificação, a Random Forest usa a votação majoritária: cada árvore faz uma previsão e a classe mais votada é escolhida como a previsão final.
- Para regressão, a Random Forest faz a média das previsões das várias árvores.

In [ ]:
#import do random forest
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#treinamento do random forest
forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(df_orders_prepared, df_orders_treatment_label)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(n_estimators=10, random_state=42)

In [ ]:
#Realiza as previsões e testa a métrica do RMSE
dados_predictions = forest_reg.predict(dados_preparados)
forest_mse = mean_squared_error(labels, dados_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

1.2915107432770352

In [ ]:
df_orders_treatment.shape

(53299, 12)

In [ ]:
forest_reg.feature_importances_.shape

(15,)

In [ ]:
#predições
dados_predictions

array([ 7.2,  8.3,  1.6,  6.5, 10.8,  1.5,  2.6,  4.5,  3. ,  2.8])

In [ ]:
#valores reais
labels.values

array([[7],
       [8],
       [1],
       [3],
       [9],
       [1],
       [3],
       [5],
       [3],
       [3]])

In [ ]:
#Nova variáveil com o R²
r_squared = r2_score(labels, dados_predictions)

# Exibindo o resultado
print(f"R²: {r_squared}")

R²: 0.7686546463245492


## Modelo Final

Agora que já definimos o nosso modelo com a melhor métrica, podemos fazer criar o modelo de Machine Learning

In [ ]:
#seleciona o dataset de teste
df_orders_test = test_set

In [ ]:
#seleciona o modelo final
final_model = forest_reg

In [ ]:
# Usando o conjunto de teste para a avaliação
X_test = df_orders_test.drop("demanda", axis=1)  # Remover a coluna 'demanda' do conjunto de teste
y_test = df_orders_test["demanda"].copy()        # Alvo verdadeiro do conjunto de teste

In [ ]:
# Transformando os dados de teste com o pipeline usado no treinamento
X_test_prepared = full_pipeline.transform(X_test)

In [ ]:
# Fazendo previsões com o modelo final
final_predictions = final_model.predict(X_test_prepared)

In [ ]:
# Calculando as métricas de desempenho
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)  # Raiz do erro quadrático médio
final_mae = mean_absolute_error(y_test, final_predictions)  # Erro absoluto médio
final_r_squared = r2_score(y_test, final_predictions) #R²

# Exibindo o resultado


# Exibindo os resultados
print(f"RMSE: {final_rmse}")
print(f"MAE: {final_mae}")
print(f"R²: {final_r_squared}")

RMSE: 5.440545881244703
MAE: 2.7298951844903065
R²: 0.6001878554921689


In [ ]:
y_test.describe()

,demanda
count,13325.000000
mean,5.001801
std,8.604602
min,1.000000
25%,1.000000
50%,2.000000
75%,5.000000
max,146.000000


In [ ]:
#avalia os dados de entrada
X_test.iloc[3]

,7818
store_id,295
channel_id,35
order_created_day,5
day_of_week,4
faixa_horario,tarde
order_metric_collected_time,2.67
order_metric_paused_time,33.22
order_metric_production_time,55.02
order_metric_walking_time,4.75
order_metric_expediton_speed_time,37.97


In [ ]:
#o valor real
y_test.iloc[3]

1

In [ ]:
#previsão final
final_predictions[3]

1.0

## Export do Modelo

No contexto de aprendizado de máquina, exportar um modelo refere-se ao processo de salvar o estado de um modelo treinado em um arquivo, permitindo que ele seja reutilizado posteriormente sem a necessidade de re-treinamento.

In [ ]:
#import do pickle
import pickle

In [ ]:
#realiza o dumping do modelo
pkl_filename = "orders_model.pkl"
pickle.dump(final_model, open(pkl_filename, 'wb'))

## Import do Modelo

In [ ]:
#import do modelo
loaded_model = pickle.load(open("orders_model.pkl", 'rb'))

In [ ]:
#faz a predição do X_test
final_predictions_pickle = loaded_model.predict(X_test_prepared)

In [ ]:
#traz as métricas do rmse e mae
final_mse = mean_squared_error(y_test, final_predictions_pickle)
finalp_rmse = np.sqrt(final_mse)

finalp_mae = mean_absolute_error(y_test, final_predictions_pickle)

final_r_squared = r2_score(y_test, final_predictions_pickle) #R²

print("RMSE: " + str(finalp_rmse))
print("MAE: " + str(finalp_mae))
print("R²: " + str(final_r_squared))

RMSE: 5.440545881244703
MAE: 2.7298951844903065
R²: 0.6001878554921689
